<div style="background-color:#4E79A7; padding: 20px; border-radius: 15px;">
    <h1 style="color:#FF7F0E; font-family: 'Roboto', sans-serif; font-size: 36px; font-weight: bold; text-align: center;">Data Cleaning Notebook for Credit Score Dataset</h1>
</div>

In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import re  # library for regex

import warnings    # Applies a warning filter to not show warning messages.
warnings.filterwarnings("ignore")  # This prevents warning messages from appearing when you run the code.
warnings.warn("this will not show") 

plt.rcParams["figure.figsize"] = (10,6)
# Sets the plot dimensions (figure size) in the Matplotlib library.
# Here (10,6) specifies a frame size of 10 units in width and 6 units in height.

sns.set_style("whitegrid")
# Sets the grid view in the Seaborn library to a white background.

pd.set_option('display.float_format', lambda x: '%.3f' % x)
# sets the float format in the pandas library.
# Here, the display format of decimal numbers is set using the lambda function.

pd.set_option('display.max_rows', None)
# sets the display.max_rows option to None if you do not want all rows in the data frames to be displayed.

pd.set_option('display.max_columns', None)
# sets the display.max_columns option to None if you do not want all columns in the data frames to be displayed.



In [2]:
train_data= pd.read_csv("/kaggle/input/credit-score-classification/train.csv")
test_data = pd.read_csv ("/kaggle/input/credit-score-classification/test.csv")

In [3]:
print(train_data.shape)
print(test_data.shape)

(100000, 28)
(50000, 27)


# ***Dataset info***<br><br>
- ***ID:*** Unique identifier for each entry in the dataset.<br>

- ***Customer_ID:*** Identifier for each customer.<br>

- ***Month:*** Month of data collection.<br>

- ***Name:*** Name of the customer.<br>

- ***Age:*** Age of the customer.<br>

- ***SSN:*** Social Security Number of the customer.<br>

- ***Occupation:*** Occupation of the customer.<br>

- ***Annual_Income:*** Annual income of the customer.<br>

- ***Monthly_Inhand_Salary:*** Monthly salary after deductions.<br>

- ***Num_Bank_Accounts:*** Number of bank accounts the customer has.<br>

- ***Num_Credit_Card:*** Number of credit cards the customer has.<br>

- ***Interest_Rate:*** Interest rate applied on loans.<br>

- ***Num_of_Loan:*** Number of loans the customer has.<br>

- ***Type_of_Loan:*** Type of loan taken by the customer.<br>

- ***Delay_from_due_date:*** Number of days delayed from due date for payments.<br>

- ***Num_of_Delayed_Payment:*** Number of delayed payments made by the customer.<br>

- ***Changed_Credit_Limit:*** Indicates if the credit limit has been changed.<br>

- ***Num_Credit_Inquiries:*** Number of credit inquiries made by the customer.<br>

- ***Credit_Mix:*** Mix of different types of credit accounts held by the customer.<br>

- ***Outstanding_Debt:*** Amount of outstanding debt.<br>

- ***Credit_Utilization_Ratio:*** Ratio of credit used to credit available.<br>

- ***Credit_History_Age:*** Age of credit history.<br>

- ***Payment_of_Min_Amount:*** Indicates if minimum payment amount is met.<br>

- ***Total_EMI_per_month:*** Total Equated Monthly Installment (EMI) paid by the customer.<br>

- ***Amount_invested_monthly:*** Amount invested monthly by the customer.<br>

- ***Payment_Behaviour:*** Payment behavior of the customer.<br>

- ***Monthly_Balance:*** Monthly balance in the account.<br>

- ***Credit_Score:*** Target variable - credit score of the customer.<br>



### Defination of df 

In [4]:
df = train_data.copy()
df.head()

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.000,_,809.98,26.823,22 Years and 1 Months,No,49.575,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.000,Good,809.98,31.945,NaN,No,49.575,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.000,Good,809.98,28.609,22 Years and 3 Months,No,49.575,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.000,Good,809.98,31.378,22 Years and 4 Months,No,49.575,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.000,Good,809.98,24.797,22 Years and 5 Months,No,49.575,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good


In [5]:
df.shape

(100000, 28)

In [6]:
df.columns

Index(['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation',
       'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age',
       'Payment_of_Min_Amount', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance',
       'Credit_Score'],
      dtype='object')

#### First of all, it would be useful to know how the relavant persons calculate the credit score and which parameters are included in the calculation of the credit score.
https://www.investopedia.com/ask/answers/05/creditscorecalculation.asp<br>
A credit score is designed to measure your risk as a borrower. FICO does not reveal its proprietary credit score calculator formula, but the calculation incorporates five major components, with varying levels of importance. These categories with their relative weights are<br>
- Payment history (35%)<br>
- Amount owed (30%)<br>
- Length of credit history (15%)<br>
- New credit (10%)<br>
- Credit mix (10%)<br><br>
***All of these categories are taken into account in the calculation of your overall score, which can range from 300 to 850.And I tried to associate the features in the dataset with the parameters in these 5 categories.***<br><br>
- Payment history (35%) and features in our dataset that may be related : **"Payment_of_Min_Amount","Payment_Behaviour","Num_of_Delayed_Payment",
"Delay_from_due_date","Outstanding_Debt","Amount_invested_monthly"** <br>
- Amount owed (30%) and features in our dataset that may be related:  **"Credit_Utilization_Ratio","Outstanding_Debt","Monthly_Balance","Amount_invested_monthly"**<br>
- Length of credit history (15%) and features in our dataset that may be related : ***"Credit_History_Age","Num_of_Loan"***   <br>
- New credit (10%) and features in our dataset that may be related : **"Num_Credit_Inquiries","Num_of_Loan","Payment_of_Min_Amount","Monthly_Balance","Amount_invested_monthly"**   <br>
- Credit mix (10%) and features in our dataset that may be related :**"Credit_Mix"**     <br><br>
 What Isn’t Included:<br>
 - Your credit score reflects only the information contained in your credit report. Your credit report doesn't include information like your **age**, **income**, or employment history. It also will generally not include your history with utilities such as cable and phone bills nor your rental payment history.

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

In [8]:
# The alternative code for an overview of dataset features ;

def compr_info(df, dropna=False):
    """
    Returns a dataframe consisting of datatypes, nuniques, #s of nulls head(1), most frequent item and its frequncy,
    where the column names are indices.
    """
    dt=pd.DataFrame(df.dtypes, columns=["Type"])
    dn=pd.DataFrame(df.nunique(), columns=["Nunique"])
    nonnull=pd.DataFrame(df.isnull().sum(), columns=["#of Missing"])
    firstT=df.head(1).T.rename(columns={0:"First"})
    MostFreqI=pd.DataFrame([df[x].value_counts().head(1).index[0] for x in df.columns], columns=["MostFreqItem"],index=df.columns)
    MostFreqC=pd.DataFrame([df[x].value_counts().head(1).values[0] for x in df.columns], columns=["MostFreqCount"],index=df.columns)
    return pd.concat([dt,dn,nonnull,MostFreqI,MostFreqC,firstT],axis=1)

compr_info(df)

,Type,Nunique,#of Missing,MostFreqItem,MostFreqCount,First
ID,object,100000,0,0x25fb6,1,0x1602
Customer_ID,object,12500,0,CUS_0x942c,8,CUS_0xd40
Month,object,8,0,January,12500,January
Name,object,10139,9985,Langep,44,Aaron Maashoh
Age,object,1788,0,38,2833,23
SSN,object,12501,0,#F%$D@*&8,5572,821-00-0265
Occupation,object,16,0,_______,7062,Scientist
Annual_Income,object,18940,0,20867.67,16,19114.12
Monthly_Inhand_Salary,float64,13235,15002,6358.957,15,1824.843
Num_Bank_Accounts,int64,943,0,6,13001,3


In [9]:
df.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit            0
Num_Credit_Inquiries         1965
Credit_Mix                      0
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [10]:
 df.isna().sum().sum()

60071

#### ***Although the columns are not NaN, there may be rows containing any other expressions that express absence.***

In [11]:
pip install colorama


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [12]:
# for checking all dataframe if there is any match with our regex pattern

pattern = re.compile(r"^[^a-zA-Z0-9 .,]+$") 

from colorama import init, Fore, Back, Style

def matched_re(df):
    total_matches = 0
    for column in df.columns:
        column_matches = 0
        matched_values = set()  
        for value in df[column]:
            if re.search(r"^[^a-zA-Z0-9 .,]+$", str(value)):
                column_matches += 1
                matched_values.add(value)
                total_matches += 1
        if column_matches > 0:
            print(f"{Back.YELLOW + Fore.BLACK} Matched values in column {column}: {column_matches} times '{', '.join(matched_values)}' {Style.RESET_ALL}")
            
        else:
            print(f"Total matches in column {column}: {column_matches}")
    print(f"{Back.YELLOW + Fore.BLACK}Total matches in dataframe: {total_matches} {Style.RESET_ALL}")

In [13]:
matched_re(df)

Total matches in column ID: 0
Total matches in column Customer_ID: 0
Total matches in column Month: 0


Total matches in column Name: 0
Total matches in column Age: 0
Total matches in column SSN: 0


 Matched values in column Occupation: 7062 times '_______' 
Total matches in column Annual_Income: 0


Total matches in column Monthly_Inhand_Salary: 0
Total matches in column Num_Bank_Accounts: 0
Total matches in column Num_Credit_Card: 0


Total matches in column Interest_Rate: 0
Total matches in column Num_of_Loan: 0


...

 Matched values in column Changed_Credit_Limit: 2091 times '_' 
Total matches in column Num_Credit_Inquiries: 0
 Matched values in column Credit_Mix: 20195 times '_' 


Total matches in column Outstanding_Debt: 0
Total matches in column Credit_Utilization_Ratio: 0


Total matches in column Credit_History_Age: 0
Total matches in column Payment_of_Min_Amount: 0


Total matches in column Total_EMI_per_month: 0
Total matches in column Amount_invested_monthly: 0


Total matches in column Payment_Behaviour: 0
Total matches in column Monthly_Balance: 0
Total matches in column Credit_Score: 0
Total matches in dataframe: 29348 


In [14]:
df.isna().sum().sum() 

60071

In [15]:
60071+29348

89419

#### *The total number of nulls is 60071 + 29348 = 89419 so many that we have a problem to overcome,first I will replace the values that are not entered and appear as underscores and similar symbols with NaN to make them easier to use.*

In [16]:
df.loc[(df["Changed_Credit_Limit"].str.contains(pattern,regex=True)==True)].head(3)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.000,Good,809.98,28.609,22 Years and 3 Months,No,49.575,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
65,0x1663,CUS_0x4157,February,Charlie Zhur,23,070-19-1622,Doctor,114838.41,9843.868,2,5,7,3,"Personal Loan, Debt Consolidation Loan, and Au...",14,8,_,3.000,Good,1377.74,27.626,21 Years and 5 Months,NM,226.893,212.23560220847847,High_spent_Large_value_payments,785.2583558699787,Good
66,0x1664,CUS_0x4157,March,Charlie Zhur,23,070-19-1622,Doctor,114838.41_,NaN,2,5,7,3,"Personal Loan, Debt Consolidation Loan, and Au...",11,11,_,3.000,Good,1377.74,35.142,21 Years and 6 Months,NM,226.893,470.3857956796373,High_spent_Small_value_payments,547.1081623988198,Good


In [17]:
df.loc[(df["Changed_Credit_Limit"].str.contains(pattern,regex=True)==True),"Changed_Credit_Limit"] = np.nan

In [18]:
df.loc[(df["Changed_Credit_Limit"].str.contains(pattern,regex=True)==True),"Changed_Credit_Limit"].count()

0

In [19]:
df.Occupation.value_counts(dropna=False)

Occupation
_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: count, dtype: int64

In [20]:
# Again for checking if the created pattern matches in only Occupation feature

for text in df.Occupation.value_counts().index:
    if re.search(pattern, text):
        print("Matched value:", text)
    else:
        print("Unmatched value:", text)

Matched value: _______
Unmatched value: Lawyer
Unmatched value: Architect
Unmatched value: Engineer
Unmatched value: Scientist
Unmatched value: Mechanic
Unmatched value: Accountant
Unmatched value: Developer
Unmatched value: Media_Manager
Unmatched value: Teacher
Unmatched value: Entrepreneur
Unmatched value: Doctor
Unmatched value: Journalist
Unmatched value: Manager
Unmatched value: Musician
Unmatched value: Writer


In [21]:
df['Occupation'] = df['Occupation'].replace(pattern,np.nan, regex=True)

In [22]:
df.Occupation.value_counts(dropna=False)

Occupation
NaN              7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: count, dtype: int64

In [23]:
df['Credit_Mix'].value_counts(dropna=False)

Credit_Mix
Standard    36479
Good        24337
_           20195
Bad         18989
Name: count, dtype: int64

In [24]:
df['Credit_Mix'] = df['Credit_Mix'].replace(pattern,np.nan, regex=True)

In [25]:
df['Credit_Mix'].value_counts(dropna=False)

Credit_Mix
Standard    36479
Good        24337
NaN         20195
Bad         18989
Name: count, dtype: int64

In [26]:
matched_re(df) # let's double check all dataframe for empty value

Total matches in column ID: 0
Total matches in column Customer_ID: 0
Total matches in column Month: 0


Total matches in column Name: 0
Total matches in column Age: 0
Total matches in column SSN: 0


Total matches in column Occupation: 0
Total matches in column Annual_Income: 0


Total matches in column Monthly_Inhand_Salary: 0
Total matches in column Num_Bank_Accounts: 0
Total matches in column Num_Credit_Card: 0


Total matches in column Interest_Rate: 0
Total matches in column Num_of_Loan: 0


...

Total matches in column Changed_Credit_Limit: 0
Total matches in column Num_Credit_Inquiries: 0
Total matches in column Credit_Mix: 0


Total matches in column Outstanding_Debt: 0
Total matches in column Credit_Utilization_Ratio: 0


Total matches in column Credit_History_Age: 0
Total matches in column Payment_of_Min_Amount: 0


Total matches in column Total_EMI_per_month: 0
Total matches in column Amount_invested_monthly: 0


Total matches in column Payment_Behaviour: 0
Total matches in column Monthly_Balance: 0
Total matches in column Credit_Score: 0
Total matches in dataframe: 0 


In [27]:
df.isna().sum()

ID                              0
Customer_ID                     0
Month                           0
Name                         9985
Age                             0
SSN                             0
Occupation                   7062
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts               0
Num_Credit_Card                 0
Interest_Rate                   0
Num_of_Loan                     0
Type_of_Loan                11408
Delay_from_due_date             0
Num_of_Delayed_Payment       7002
Changed_Credit_Limit         2091
Num_Credit_Inquiries         1965
Credit_Mix                  20195
Outstanding_Debt                0
Credit_Utilization_Ratio        0
Credit_History_Age           9030
Payment_of_Min_Amount           0
Total_EMI_per_month             0
Amount_invested_monthly      4479
Payment_Behaviour               0
Monthly_Balance              1200
Credit_Score                    0
dtype: int64

In [28]:
df.columns = df.columns.str.lower()
df.columns

Index(['id', 'customer_id', 'month', 'name', 'age', 'ssn', 'occupation',
       'annual_income', 'monthly_inhand_salary', 'num_bank_accounts',
       'num_credit_card', 'interest_rate', 'num_of_loan', 'type_of_loan',
       'delay_from_due_date', 'num_of_delayed_payment', 'changed_credit_limit',
       'num_credit_inquiries', 'credit_mix', 'outstanding_debt',
       'credit_utilization_ratio', 'credit_history_age',
       'payment_of_min_amount', 'total_emi_per_month',
       'amount_invested_monthly', 'payment_behaviour', 'monthly_balance',
       'credit_score'],
      dtype='object')

In [29]:
df.shape

(100000, 28)

In [30]:
null_columns = df.columns[df.isnull().any()]
df[null_columns].isnull().sum().sort_values(ascending=False)

credit_mix                 20195
monthly_inhand_salary      15002
type_of_loan               11408
name                        9985
credit_history_age          9030
occupation                  7062
num_of_delayed_payment      7002
amount_invested_monthly     4479
changed_credit_limit        2091
num_credit_inquiries        1965
monthly_balance             1200
dtype: int64

In [31]:
df["credit_mix"].value_counts(dropna=False)

credit_mix
Standard    36479
Good        24337
NaN         20195
Bad         18989
Name: count, dtype: int64

In [32]:
df[df["credit_mix"].isna()][null_columns].isna().sum().sort_values(ascending=False)

credit_mix                 20195
monthly_inhand_salary       3042
type_of_loan                2408
name                        2028
credit_history_age          1824
occupation                  1421
num_of_delayed_payment      1388
amount_invested_monthly      861
changed_credit_limit         406
num_credit_inquiries         396
monthly_balance              236
dtype: int64

#### *Based on the above output, we can say that the values in 10 columns are missing for 236 rows in the data set.*

#### ***The codes below are to investigate whether I can benefit from the information in other columns on how to fill in the missing data.***

In [33]:
indices = df[null_columns].isnull().sum().sort_values(ascending=False).index

In [34]:
col_total = df[null_columns].isnull().sum().sort_values(ascending=False).values.tolist()
based_on_cr_mix =df[df["credit_mix"].isna()][null_columns].isna().sum().sort_values(ascending=False).values.tolist()
rate = []
for i,j in zip(col_total,based_on_cr_mix):
    rate_of_missing = j/i
    rate.append(rate_of_missing)
missing_data = pd.DataFrame({"total_null_count": col_total, "based_on_credit_mix": based_on_cr_mix, "rate": rate},index=indices)

missing_data

,total_null_count,based_on_credit_mix,rate
credit_mix,20195,20195,1.000
monthly_inhand_salary,15002,3042,0.203
type_of_loan,11408,2408,0.211
name,9985,2028,0.203
credit_history_age,9030,1824,0.202
occupation,7062,1421,0.201
num_of_delayed_payment,7002,1388,0.198
amount_invested_monthly,4479,861,0.192
changed_credit_limit,2091,406,0.194
num_credit_inquiries,1965,396,0.202


In [35]:
df["credit_mix"].isna().sum()

20195

In [36]:
df[df["customer_id"]=="CUS_0x10fd"]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
25456,0xab2a,CUS_0x10fd,January,Hendersonu,29,119-33-7350,NaN,39576.59,3199.049,0,7,2,4,"Personal Loan, Mortgage Loan, Home Equity Loan...",18,NaN,4.62,1.000,Good,226.37,23.086,19 Years and 4 Months,No,124.311,52.588915433114344,High_spent_Large_value_payments,383.0051621399922,Poor
25457,0xab2b,CUS_0x10fd,February,Hendersonu,29,119-33-7350,Developer,39576.59_,3199.049,0,7,2,4_,"Personal Loan, Mortgage Loan, Home Equity Loan...",14,4,10.62,1.000,NaN,226.37,31.941,19 Years and 5 Months,NM,124.311,28.89323934124009,High_spent_Large_value_payments,406.7008382318664,Poor
25458,0xab2c,CUS_0x10fd,March,Hendersonu,29_,119-33-7350,Developer,39576.59,3199.049,0,7,2,4,"Personal Loan, Mortgage Loan, Home Equity Loan...",14,4,-2.38,1.000,NaN,226.37,33.295,19 Years and 6 Months,No,124.311,176.5989160690919,Low_spent_Large_value_payments,288.9951615040146,Poor
25459,0xab2d,CUS_0x10fd,April,Hendersonu,29,119-33-7350,Developer,39576.59,NaN,0,7,2,4,"Personal Loan, Mortgage Loan, Home Equity Loan...",14,4,8.620000000000001,1.000,Good,226.37,39.889,NaN,No,124.311,81.30606467113599,High_spent_Medium_value_payments,364.2880129019706,Poor
25460,0xab2e,CUS_0x10fd,May,Hendersonu,29_,119-33-7350,Developer,39576.59,3199.049,0,7,2,4,"Personal Loan, Mortgage Loan, Home Equity Loan...",14,1,4.62,1.000,Good,226.37,35.163,19 Years and 8 Months,No,124.311,269.0649060320007,Low_spent_Large_value_payments,196.52917154110585,Poor
25461,0xab2f,CUS_0x10fd,June,Hendersonu,29,119-33-7350,Developer,39576.59,3199.049,0,7,2,4,"Personal Loan, Mortgage Loan, Home Equity Loan...",14,6,4.62,1.000,Good,226.37,29.267,19 Years and 9 Months,No,124.311,162.99845492614182,Low_spent_Medium_value_payments,312.59562264696467,Poor
25462,0xab30,CUS_0x10fd,July,Hendersonu,29,119-33-7350,Developer,39576.59,3199.049,0,7,2,4,"Personal Loan, Mortgage Loan, Home Equity Loan...",11,4,6.62,3.000,Good,226.37,24.766,19 Years and 10 Months,No,124.311,199.44446359793903,Low_spent_Medium_value_payments,276.14961397516737,Poor
25463,0xab31,CUS_0x10fd,August,Hendersonu,29,119-33-7350,Developer,39576.59,3199.049,0,7,2,4,"Personal Loan, Mortgage Loan, Home Equity Loan...",14,4,4.62,3.000,NaN,226.37,29.326,19 Years and 11 Months,No,124.311,104.7631642205206,High_spent_Medium_value_payments,340.83091335258587,Poor


In [37]:
df["customer_id"].nunique()

12500

#### *Yes! I can immediately access the information I want from the "customer_id" feature. There are 12500 unique customer_ids in the 100000 rows data set. Since there is no NaN value in this column, this means that  it repeats and I can fill in the NaN values in the rows with the same customer_id with those in the full rows.*

In [38]:
unique_customer_ids = df["customer_id"].unique()

In [39]:
for id in unique_customer_ids :
    df[df["customer_id"]==id] = df[df["customer_id"]==id].fillna(method='ffill').fillna(method='bfill')

In [40]:
df.isna().sum()

id                              0
customer_id                     0
month                           0
name                            0
age                             0
ssn                             0
occupation                      0
annual_income                   0
monthly_inhand_salary           0
num_bank_accounts               0
num_credit_card                 0
interest_rate                   0
num_of_loan                     0
type_of_loan                11408
delay_from_due_date             0
num_of_delayed_payment          0
changed_credit_limit            0
num_credit_inquiries            0
credit_mix                      0
outstanding_debt                0
credit_utilization_ratio        0
credit_history_age              0
payment_of_min_amount           0
total_emi_per_month             0
amount_invested_monthly         0
payment_behaviour               0
monthly_balance                 0
credit_score                    0
dtype: int64

In [41]:
df[df["customer_id"]=="CUS_0xd40"]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.000,Good,809.98,26.823,22 Years and 1 Months,No,49.575,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,7,11.27,4.000,Good,809.98,31.945,22 Years and 1 Months,No,49.575,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.000,Good,809.98,28.609,22 Years and 3 Months,No,49.575,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.000,Good,809.98,31.378,22 Years and 4 Months,No,49.575,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,4,11.27,4.000,Good,809.98,24.797,22 Years and 5 Months,No,49.575,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
5,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",8,4,9.27,4.000,Good,809.98,27.262,22 Years and 6 Months,No,49.575,62.430172331195294,!@9#%8,340.4792117872438,Good
6,0x1608,CUS_0xd40,July,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,8_,11.27,4.000,Good,809.98,22.538,22 Years and 7 Months,No,49.575,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
7,0x1609,CUS_0xd40,August,Aaron Maashoh,23,#F%$D@*&8,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,6,11.27,4.000,Good,809.98,23.934,22 Years and 7 Months,No,49.575,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard


In [42]:
df[df["customer_id"]=="CUS_0x93ff"]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
1584,0x1f4a,CUS_0x93ff,January,Roumeliotisv,26,237-50-9740,Lawyer,39696.98,3211.082,5,5,10,1,Not Specified,16,3,1.98,1.000,Good,133.3,39.886,21 Years and 3 Months,No,31.907,73.87093984450794,High_spent_Medium_value_payments,465.33064665134935,Poor
1585,0x1f4b,CUS_0x93ff,February,Roumeliotisv,26,237-50-9740,Lawyer,39696.98_,3211.082,5,5,10,1_,Not Specified,11,5,-5.02,1.000,Good,133.3,29.998,21 Years and 3 Months,NM,31.907,67.0732807694454,High_spent_Medium_value_payments,472.12830572641184,Poor
1586,0x1f4c,CUS_0x93ff,March,Roumeliotisv,26,237-50-9740,Lawyer,39696.98,3211.082,5,5,10,1,Not Specified,11,3,1.98,1.000,Good,133.3,33.396,21 Years and 5 Months,No,31.907,56.2457675369253,High_spent_Medium_value_payments,482.9558189589319,Standard
1587,0x1f4d,CUS_0x93ff,April,Roumeliotisv,26,237-50-9740,Lawyer,39696.98,3211.082,5,5,10,1,Not Specified,11,3,1.98,1.000,Good,133.3,28.844,21 Years and 6 Months,No,31.907,37.51782482904488,High_spent_Medium_value_payments,501.6837616668123,Poor
1588,0x1f4e,CUS_0x93ff,May,Roumeliotisv,26,237-50-9740,Lawyer,39696.98,3211.082,5,5,1115,1,Not Specified,11,6,1.98,1.000,Good,133.3,40.143,21 Years and 7 Months,No,31.907,44.04731679197315,High_spent_Large_value_payments,485.154269703884,Poor
1589,0x1f4f,CUS_0x93ff,June,Roumeliotisv,26,237-50-9740,Lawyer,39696.98,3211.082,5,5,10,1,Not Specified,11,3,1.98,1.000,Good,133.3,24.870,21 Years and 8 Months,No,31.907,156.79219440803175,Low_spent_Small_value_payments,422.4093920878255,Poor
1590,0x1f50,CUS_0x93ff,July,Roumeliotisv,27,237-50-9740,Lawyer,39696.98,3211.082,5,5,10,1,Not Specified,11,3,1.98,1.000,Good,133.3,31.677,21 Years and 9 Months,No,31.907,223.97377228626283,Low_spent_Large_value_payments,335.22781420959444,Poor
1591,0x1f51,CUS_0x93ff,August,Roumeliotisv,27,237-50-9740,Lawyer,39696.98,3211.082,5,5,10,1,Not Specified,11,3,1.98,1194.000,Good,133.3,27.290,21 Years and 10 Months,No,31.907,239.0805687710546,Low_spent_Medium_value_payments,330.1210177248025,Poor


In [43]:
df[["type_of_loan"]].sample(20)

,type_of_loan
48355,"Credit-Builder Loan, Auto Loan, Credit-Builder..."
34886,"Auto Loan, and Personal Loan"
24325,"Home Equity Loan, and Auto Loan"
30446,"Payday Loan, Payday Loan, Debt Consolidation L..."
66840,NaN
53690,"Student Loan, Home Equity Loan, Home Equity Lo..."
90549,"Auto Loan, Mortgage Loan, Personal Loan, and H..."
52632,"Student Loan, and Home Equity Loan"
98218,"Auto Loan, Debt Consolidation Loan, and Debt C..."
67395,Debt Consolidation Loan


In [44]:
# Function used to transform the "credit history age" column to simplify our dataset and use numerical data
# I changed the information given in "15 years and 5 Months" format as year and month to the month equivalent, that is, "185 Months" format.
def credit_history(duration):                        
    years, months = duration.split(' Years and ')
    months = months.split(' Months')[0]
    total_months = int(years) * 12 + int(months)
    return total_months

In [45]:
df["credit_history_age"] = df["credit_history_age"].apply(credit_history) 
df["credit_history_age"].head()

0    265
1    265
2    267
3    268
4    269
Name: credit_history_age, dtype: int64

In [46]:
df.sample(5)

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
57196,0x16522,CUS_0x970f,May,Hodog,30,#F%$D@*&8,Musician,121944.2,10247.017,3,6,2,1,Not Specified,23,17,9.16,2.000,Good,1344.25,35.793,362,No,50685.000,229.845873287016,High_spent_Large_value_payments,978.8201936792681,Standard
548,0x1936,CUS_0x74f2,May,Colew,30,755-18-0308,Architect,30390.44_,2721.537,5,6,5,4,"Mortgage Loan, Student Loan, Not Specified, an...",15,18,8.36,4.000,Standard,580.31,38.002,295,No,64504.000,169.87637431078454,Low_spent_Large_value_payments,307.42925415086404,Standard
30170,0xc6c8,CUS_0xc2e8,March,Blamontc,40,243-24-5828,Architect,38325.88,2942.823,3,7,22,7,"Auto Loan, Personal Loan, Personal Loan, Credi...",10,20,15.88,7.000,Standard,1765.56,24.139,113,Yes,173.695,297.7419506886757,Low_spent_Small_value_payments,112.84534812275179,Poor
18191,0x8095,CUS_0x8d45,August,Terril Yued,38,239-07-9719,Mechanic,18718.52,1784.877,6,9,16,-100,"Credit-Builder Loan, Student Loan, Credit-Buil...",56,15,18.73,14.000,Bad,4432.96,33.606,139,Yes,126.491,47.587900225325754,High_spent_Medium_value_payments,254.40827721361592,Poor
62533,0x18467,CUS_0x3033,June,Tommyl,55,159-13-7415,Teacher,97635.87,6907.982,3,4,12,-100,"Student Loan, Personal Loan, and Auto Loan",18,8,7.17,4.000,Good,1298.97,31.720,309,No,1203.206,131.8406790245576,High_spent_Large_value_payments,684.0259203446677,Standard


In [47]:
df.age = df.age.apply(lambda x : x.rstrip("_").lstrip("_"))

In [48]:
df["amount_invested_monthly"]=df["amount_invested_monthly"].apply(lambda x : x.rstrip("__").lstrip("__"))

In [49]:
df["amount_invested_monthly"]=df["amount_invested_monthly"].apply(lambda x : round(float(x),3))

In [50]:
df["monthly_balance"]=df["monthly_balance"].apply(lambda x : str(x).lstrip("__").rstrip("__"))

In [51]:
df["monthly_balance"]=df["monthly_balance"].apply(lambda x : round(float(x),3))

In [52]:
df.head()

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.000,Good,809.98,26.823,265,No,49.575,80.415,High_spent_Small_value_payments,312.494,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,7,11.27,4.000,Good,809.98,31.945,265,No,49.575,118.280,Low_spent_Large_value_payments,284.629,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.000,Good,809.98,28.609,267,No,49.575,81.700,Low_spent_Medium_value_payments,331.210,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.000,Good,809.98,31.378,268,No,49.575,199.458,Low_spent_Small_value_payments,223.451,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,4,11.27,4.000,Good,809.98,24.797,269,No,49.575,41.420,High_spent_Medium_value_payments,341.489,Good


In [53]:
compr_info(df)

,Type,Nunique,#of Missing,MostFreqItem,MostFreqCount,First
id,object,100000,0,0x25fb6,1,0x1602
customer_id,object,12500,0,CUS_0x942c,8,CUS_0xd40
month,object,8,0,January,12500,January
name,object,10139,0,Stevex,48,Aaron Maashoh
age,object,1728,0,38,2994,23
ssn,object,12501,0,#F%$D@*&8,5572,821-00-0265
occupation,object,15,0,Lawyer,7096,Scientist
annual_income,object,18940,0,20867.67,16,19114.12
monthly_inhand_salary,float64,13235,0,3080.555,16,1824.843
num_bank_accounts,int64,943,0,6,13001,3


### Monthly_balance feature

In [54]:
df["monthly_balance"].describe()

count                         100000.000
mean        -29999999999999995805696.000
std        3162151165267075223519232.000
min     -333333333333333314856026112.000
25%                              270.016
50%                              337.068
75%                              471.533
max                             1602.041
Name: monthly_balance, dtype: float64

In [55]:
df.shape

(100000, 28)

In [56]:
q1 = 270.035
q3 = 471.606
iqr = q3 - q1 
lower_limit = q1-(2 * iqr)
top_limit = df["monthly_balance"].max()

In [57]:
lower_limit

-133.1069999999999

In [58]:
top_limit

1602.041

In [59]:
 df.loc[(df["monthly_balance"] < lower_limit)][["customer_id","month","monthly_balance"]]

,customer_id,month,monthly_balance
5545,CUS_0x9885,February,-333333333333333314856026112.000
26177,CUS_0x5a90,February,-333333333333333314856026112.000
29158,CUS_0x288d,July,-333333333333333314856026112.000
35570,CUS_0x85e9,March,-333333333333333314856026112.000
38622,CUS_0x2b77,July,-333333333333333314856026112.000
60009,CUS_0xc06e,February,-333333333333333314856026112.000
75251,CUS_0x57f3,April,-333333333333333314856026112.000
82918,CUS_0x41bf,July,-333333333333333314856026112.000
83255,CUS_0x2f7e,August,-333333333333333314856026112.000


In [60]:
df.loc[df["customer_id"]=="CUS_0x9885"][["customer_id","month","monthly_balance"]]

,customer_id,month,monthly_balance
5544,CUS_0x9885,January,423.397
5545,CUS_0x9885,February,-333333333333333314856026112.000
5546,CUS_0x9885,March,278.412
5547,CUS_0x9885,April,420.557
5548,CUS_0x9885,May,71.288
5549,CUS_0x9885,June,383.284
5550,CUS_0x9885,July,229.007
5551,CUS_0x9885,August,374.031


In [61]:
df.loc[((df["customer_id"]=="CUS_0x9885") & (df["month"]=="February")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x9885")&(df["month"]!="February")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0x5a90") & (df["month"]=="February")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x5a90")&(df["month"]!="February")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0x288d") & (df["month"]=="July")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x288d")&(df["month"]!="July")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0x85e9") & (df["month"]=="March")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x85e9")&(df["month"]!="March")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0x2b77") & (df["month"]=="July")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x2b77")&(df["month"]!="July")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0xc06e") & (df["month"]=="February")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0xc06e")&(df["month"]!="February")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0x57f3") & (df["month"]=="April")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x57f3")&(df["month"]!="April")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0x41bf") & (df["month"]=="July")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x41bf")&(df["month"]!="July")]["monthly_balance"].mean()
df.loc[((df["customer_id"]=="CUS_0x2f7e") & (df["month"]=="August")),"monthly_balance"] = df.loc[(df["customer_id"]=="CUS_0x2f7e")&(df["month"]!="August")]["monthly_balance"].mean()

In [62]:
 df.loc[[5545,26177,29158,35570,38622,60009,75251,82918,83255]][["customer_id","month","monthly_balance"]]

,customer_id,month,monthly_balance
5545,CUS_0x9885,February,311.425
26177,CUS_0x5a90,February,333.956
29158,CUS_0x288d,July,283.236
35570,CUS_0x85e9,March,319.296
38622,CUS_0x2b77,July,336.442
60009,CUS_0xc06e,February,702.613
75251,CUS_0x57f3,April,506.502
82918,CUS_0x41bf,July,237.225
83255,CUS_0x2f7e,August,847.945


In [63]:
df["monthly_balance"].isna().sum()

0

In [64]:
df.isna().sum()

id                              0
customer_id                     0
month                           0
name                            0
age                             0
ssn                             0
occupation                      0
annual_income                   0
monthly_inhand_salary           0
num_bank_accounts               0
num_credit_card                 0
interest_rate                   0
num_of_loan                     0
type_of_loan                11408
delay_from_due_date             0
num_of_delayed_payment          0
changed_credit_limit            0
num_credit_inquiries            0
credit_mix                      0
outstanding_debt                0
credit_utilization_ratio        0
credit_history_age              0
payment_of_min_amount           0
total_emi_per_month             0
amount_invested_monthly         0
payment_behaviour               0
monthly_balance                 0
credit_score                    0
dtype: int64

### Age Feature

In [65]:
df.age.astype("int").describe()

count   100000.000
mean       110.650
std        686.245
min       -500.000
25%         24.000
50%         33.000
75%         42.000
max       8698.000
Name: age, dtype: float64

In [66]:
df.age = df.age.astype("int")

In [67]:
df.age.dtype

dtype('int64')

In [68]:
len(df.loc[df["age"] > 56])

1895

In [69]:
len(df.loc[df["age"] < 14])

886

In [70]:
df.loc[df["age"] < 14].age.describe()

count    886.000
mean    -500.000
std        0.000
min     -500.000
25%     -500.000
50%     -500.000
75%     -500.000
max     -500.000
Name: age, dtype: float64

In [71]:
df.age.describe()

count   100000.000
mean       110.650
std        686.245
min       -500.000
25%         24.000
50%         33.000
75%         42.000
max       8698.000
Name: age, dtype: float64

In [72]:
df.loc[df["age"] == 14 ].sample(3)

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
65925,0x19847,CUS_0xac29,June,Susanq,14,985-65-5941,Doctor,9654.115,620.510,6,7,18,3,"Credit-Builder Loan, Credit-Builder Loan, and ...",57,18,14.3,10.000,Standard,2081.23,31.813,110,Yes,20.328,69.401,Low_spent_Small_value_payments,262.323,Poor
14196,0x692e,CUS_0x6958,May,Tarao,14,811-14-4092,Manager,28312.79,2566.399,3,3,5,3,"Auto Loan, Not Specified, and Mortgage Loan",29,7,16.57,5.000,Standard,365.77,32.406,115,Yes,50.677,271.931,Low_spent_Small_value_payments,224.031,Standard
48171,0x13041,CUS_0xa3db,April,Euan Rochap,14,736-80-3805,Accountant,18290.03_,1454.169,8,7,16,5,"Credit-Builder Loan, Credit-Builder Loan, Stud...",35,20_,15.61,8.000,Bad,4107.9,31.361,152,Yes,66.310,78.583,Low_spent_Small_value_payments,290.523,Poor


In [73]:
df.loc[df["age"] > 56].age.describe()

count   1895.000
mean    4363.394
std     2506.279
min       95.000
25%     2165.000
50%     4380.000
75%     6518.000
max     8698.000
Name: age, dtype: float64

In [74]:
df.loc[df["age"] == 4380.000]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
20368,0x8d5a,CUS_0x1f2e,January,Stefano Bernabeiq,4380,049-84-5378,Entrepreneur,41012.42,3341.702,5,4,27,7,"Payday Loan, Debt Consolidation Loan, Home Equ...",13,13,19.47,4.000,Standard,1690.55,26.118,88,Yes,199.368,351.341,Low_spent_Small_value_payments,73.461,Poor


In [75]:
df.loc[df["customer_id"] == "CUS_0x1f2e"]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
20368,0x8d5a,CUS_0x1f2e,January,Stefano Bernabeiq,4380,049-84-5378,Entrepreneur,41012.42,3341.702,5,4,27,7,"Payday Loan, Debt Consolidation Loan, Home Equ...",13,13,19.47,4.000,Standard,1690.55,26.118,88,Yes,199.368,351.341,Low_spent_Small_value_payments,73.461,Poor
20369,0x8d5b,CUS_0x1f2e,February,Stefano Bernabeiq,26,049-84-5378,Entrepreneur,41012.42,3341.702,5,4,27,7,"Payday Loan, Debt Consolidation Loan, Home Equ...",13,11,22.47,4.000,Standard,1690.55,26.470,89,Yes,199.368,205.382,Low_spent_Large_value_payments,199.421,Poor
20370,0x8d5c,CUS_0x1f2e,March,Stefano Bernabeiq,26,049-84-5378,Entrepreneur,41012.42,3341.702,5,4,27,7,"Payday Loan, Debt Consolidation Loan, Home Equ...",9,11,19.47,4.000,Standard,1690.55,25.467,90,NM,199.368,109.167,!@9#%8,285.636,Poor
20371,0x8d5d,CUS_0x1f2e,April,Stefano Bernabeiq,26,049-84-5378,Entrepreneur,41012.42_,3341.702,5,4,27,7_,"Payday Loan, Debt Consolidation Loan, Home Equ...",8,14,14.47,4.000,Standard,1690.55,30.387,91,NM,199.368,224.138,Low_spent_Small_value_payments,200.664,Poor
20372,0x8d5e,CUS_0x1f2e,May,Stefano Bernabeiq,26,049-84-5378,Entrepreneur,41012.42_,3341.702,5,4,27,-100,"Payday Loan, Debt Consolidation Loan, Home Equ...",13,11,19.47,4.000,Standard,1690.55,37.308,91,Yes,199.368,117.137,High_spent_Small_value_payments,277.665,Poor
20373,0x8d5f,CUS_0x1f2e,June,Stefano Bernabeiq,26,049-84-5378,Entrepreneur,41012.42,3341.702,5,4,27,7,"Payday Loan, Debt Consolidation Loan, Home Equ...",13,11,14.47,4.000,Standard,1690.55,31.645,91,Yes,199.368,118.545,High_spent_Medium_value_payments,266.257,Standard
20374,0x8d60,CUS_0x1f2e,July,Stefano Bernabeiq,26,049-84-5378,Entrepreneur,41012.42,3341.702,5,4,27,7,"Payday Loan, Debt Consolidation Loan, Home Equ...",18,10,19.47,4.000,Standard,1690.55,32.476,94,Yes,199.368,89.851,High_spent_Medium_value_payments,294.951,Poor
20375,0x8d61,CUS_0x1f2e,August,Stefano Bernabeiq,26,049-84-5378,Entrepreneur,41012.42,3341.702,5,4,27,7,"Payday Loan, Debt Consolidation Loan, Home Equ...",13,9,14.47,4.000,Standard,1690.55,30.687,95,Yes,199.368,174.795,High_spent_Small_value_payments,220.007,Standard


In [76]:
len(df[df["age"].apply(lambda x: len(str(int(x))) > 2)])

2777

In [77]:
df.loc[df["age"].apply(lambda x: len(str(int(x))) > 2),"age"]=np.nan

In [78]:
df.age.isna().sum()

2777

In [79]:
df[df["customer_id"]=="CUS_0x3861"]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
712,0x1a2e,CUS_0x3861,January,Fiona Ortizx,53.000,212-32-2085,Engineer,17992.775,1769.398,3,5,6,-100,NaN,5,9,8.61,2.000,Good,565.22,22.380,227,No,0.000,88.443,!@9#%8,348.497,Poor
713,0x1a2f,CUS_0x3861,February,Fiona Ortizx,53.000,212-32-2085,Engineer,17992.775_,1769.398,3,5,6,0,NaN,11,15,8.61,2.000,Good,565.22,37.219,228,NM,0.000,137.398,Low_spent_Small_value_payments,329.541,Poor
714,0x1a30,CUS_0x3861,March,Fiona Ortizx,NaN,212-32-2085,Engineer,17992.775,1769.398,3,5,6,0,NaN,4,12,4.6099999999999985,2.000,Good,565.22,23.137,229,No,0.000,95.656,High_spent_Small_value_payments,341.283,Poor
715,0x1a31,CUS_0x3861,April,Fiona Ortizx,53.000,#F%$D@*&8,Engineer,17992.775,1769.398,3,5,6,0,NaN,8,12,4.6099999999999985,2.000,Good,565.22,25.152,230,No,0.000,162.643,Low_spent_Small_value_payments,304.297,Poor
716,0x1a32,CUS_0x3861,May,Fiona Ortizx,53.000,212-32-2085,Engineer,17992.775,1769.398,3,5,6,0,NaN,8,10,8.61,3.000,Good,565.22,34.182,231,No,0.000,10000.000,High_spent_Medium_value_payments,386.841,Standard
717,0x1a33,CUS_0x3861,June,Fiona Ortizx,53.000,212-32-2085,Engineer,17992.775,1769.398,3,5,6,0,NaN,5,10,8.61,3.000,Good,565.22,31.054,232,No,0.000,23.003,!@9#%8,393.937,Poor
718,0x1a34,CUS_0x3861,July,Fiona Ortizx,53.000,212-32-2085,Engineer,17992.775,1769.398,3,5,6,0,NaN,6,10,8.61,3.000,Good,565.22,33.160,233,No,0.000,14.787,High_spent_Large_value_payments,402.153,Poor
719,0x1a35,CUS_0x3861,August,Fiona Ortizx,53.000,212-32-2085,Engineer,17992.775,1769.398,3,5,570,0,NaN,8,10,8.61,3.000,Good,565.22,33.464,234,No,0.000,79.040,Low_spent_Small_value_payments,387.900,Poor


In [80]:
nan_customer_ids = df.loc[df['age'].isna(), 'customer_id'].unique()

In [81]:
len(nan_customer_ids)

2526

In [82]:
for customer_id in nan_customer_ids:
    
        right_age = df.loc[(df['customer_id'] == customer_id) & (~df['age'].isna()), 'age'].iloc[0]
        df.loc[(df['customer_id'] == customer_id) & (df['age'].isna()), 'age'] = right_age

In [83]:
df.age.isna().sum()

0

In [84]:
df.age.describe()

count   100000.000
mean        33.306
std         10.772
min         14.000
25%         24.000
50%         33.000
75%         42.000
max         99.000
Name: age, dtype: float64

In [85]:
age_max_idx = df.loc[df["age"]>56].index
age_max_idx

Index([28718, 38248, 65420, 78564], dtype='int64')

In [86]:
df.drop(age_max_idx,inplace=True)

In [87]:
df.age.describe()

count   99996.000
mean       33.303
std        10.765
min        14.000
25%        24.000
50%        33.000
75%        42.000
max        56.000
Name: age, dtype: float64

### Payment Behaviour feature

In [88]:
df["payment_behaviour"].value_counts()

payment_behaviour
Low_spent_Small_value_payments      25512
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11339
Low_spent_Large_value_payments      10424
!@9#%8                               7599
Name: count, dtype: int64

In [89]:
pattern = re.compile(r"\W") 

In [90]:
# for checking all dataframe if there is any match with our regex pattern


from colorama import init, Fore, Back, Style

def matched_re(df):
    total_matches = 0
    for column in df.columns:
        column_matches = 0
        matched_values = set()  
        for value in df[column]:
            if re.search(r"^[^a-zA-Z0-9 .,]+$", str(value)):
                column_matches += 1
                matched_values.add(value)
                total_matches += 1
        if column_matches > 0:
            print(f"{Back.YELLOW + Fore.BLACK} Matched values in column {column}: {column_matches} times '{', '.join(matched_values)}' {Style.RESET_ALL}")
            
        else:
            print(f"Total matches in column {column}: {column_matches}")
    print(f"{Back.YELLOW + Fore.BLACK}Total matches in dataframe: {total_matches} {Style.RESET_ALL}")

In [91]:
for text in df["payment_behaviour"].value_counts().index:
    if re.search(pattern, text):
        print("Matched value:", text)
    else:
        print("Unmatched value:", text)

Unmatched value: Low_spent_Small_value_payments
Unmatched value: High_spent_Medium_value_payments
Unmatched value: Low_spent_Medium_value_payments
Unmatched value: High_spent_Large_value_payments
Unmatched value: High_spent_Small_value_payments
Unmatched value: Low_spent_Large_value_payments
Matched value: !@9#%8


In [92]:
df.loc[df["payment_behaviour"]=="!@9#%8"]["customer_id"].nunique()

5900

In [93]:
df.loc[df["payment_behaviour"]=="!@9#%8"]["customer_id"].sample(3)

85499    CUS_0x1628
97343    CUS_0xa015
20739    CUS_0x3e91
Name: customer_id, dtype: object

In [94]:
df.loc[df["customer_id"]=="CUS_0xc9f"]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
21320,0x92ee,CUS_0xc9f,January,Jennifert,14.000,594-51-0874,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",18,9,14.18,4.000,Standard,1059.86,28.268,82,NM,67839.000,143.294,Low_spent_Small_value_payments,284.486,Standard
21321,0x92ef,CUS_0xc9f,February,Jennifert,15.000,594-51-0874,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",23,11,14.18,4.000,Standard,1059.86,39.330,83,Yes,30.710,214.454,!@9#%8,213.326,Standard
21322,0x92f0,CUS_0xc9f,March,Jennifert,15.000,594-51-0874,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",23,9,14.18,4.000,Standard,1059.86,32.736,84,Yes,30.710,40.948,High_spent_Large_value_payments,336.832,Standard
21323,0x92f1,CUS_0xc9f,April,Jennifert,15.000,594-51-0874,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",18,11,14.18,4.000,Standard,1059.86,32.621,85,Yes,30.710,133.761,Low_spent_Small_value_payments,294.019,Standard
21324,0x92f2,CUS_0xc9f,May,Jennifert,15.000,594-51-0874,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",22,11,14.18,4.000,Standard,1059.86,32.024,86,Yes,30.710,157.990,!@9#%8,249.790,Standard
21325,0x92f3,CUS_0xc9f,June,Jennifert,14.000,594-51-0874,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",18,11,14.18,4.000,Standard,1059.86,24.024,87,Yes,65762.000,42.489,High_spent_Medium_value_payments,345.291,Standard
21326,0x92f4,CUS_0xc9f,July,Jennifert,14.000,594-51-0874,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",19,11,7.18,8.000,Standard,1059.86,25.803,88,NM,30.710,10000.000,High_spent_Large_value_payments,365.335,Standard
21327,0x92f5,CUS_0xc9f,August,Jennifert,15.000,#F%$D@*&8,Scientist,21382.81,1684.901,7,5,14,2,"Debt Consolidation Loan, and Auto Loan",17,11,18.18,8.000,Standard,1059.86,33.045,89,Yes,30.710,100.701,Low_spent_Small_value_payments,327.079,Standard


In [95]:
df.loc[df["customer_id"]=="CUS_0xc9f"]["payment_behaviour"].unique()

array(['Low_spent_Small_value_payments', '!@9#%8',
       'High_spent_Large_value_payments',
       'High_spent_Medium_value_payments'], dtype=object)

In [96]:
df.loc[df["customer_id"]=="CUS_0xc9f"]["payment_behaviour"].value_counts()

payment_behaviour
Low_spent_Small_value_payments      3
!@9#%8                              2
High_spent_Large_value_payments     2
High_spent_Medium_value_payments    1
Name: count, dtype: int64

In [97]:
df.loc[df["customer_id"]=="CUS_0xb201"]["payment_behaviour"].value_counts()

payment_behaviour
High_spent_Large_value_payments    2
!@9#%8                             2
Low_spent_Small_value_payments     2
Low_spent_Medium_value_payments    1
High_spent_Small_value_payments    1
Name: count, dtype: int64

In [98]:
undetermined_idx = df.loc[df["payment_behaviour"]=="!@9#%8"]["customer_id"].unique()

In [99]:
for customer_id in undetermined_idx:
    
        customer_ids = df[(df['customer_id'] == customer_id) & (df['payment_behaviour'] != "!@9#%8")]
        freq = customer_ids['payment_behaviour'].mode()[0]
        df.loc[(df['customer_id'] == customer_id) & (df['payment_behaviour'] == "!@9#%8"), 'payment_behaviour'] = freq
              
    

#### *I want to see the results of the changes I made*

In [100]:
df.loc[df["customer_id"]=="CUS_0xc9f"]["payment_behaviour"].unique()

array(['Low_spent_Small_value_payments',
       'High_spent_Large_value_payments',
       'High_spent_Medium_value_payments'], dtype=object)

In [101]:
df.loc[df["customer_id"]=="CUS_0xb201"]["payment_behaviour"].value_counts()

payment_behaviour
High_spent_Large_value_payments    4
Low_spent_Small_value_payments     2
Low_spent_Medium_value_payments    1
High_spent_Small_value_payments    1
Name: count, dtype: int64

In [102]:
df.loc[df["customer_id"]=="CUS_0xc9f"]["payment_behaviour"].value_counts()

payment_behaviour
Low_spent_Small_value_payments      5
High_spent_Large_value_payments     2
High_spent_Medium_value_payments    1
Name: count, dtype: int64

In [103]:
df.loc[df["customer_id"]=="CUS_0x4437"]["payment_behaviour"].value_counts()

payment_behaviour
High_spent_Medium_value_payments    5
Low_spent_Small_value_payments      2
High_spent_Small_value_payments     1
Name: count, dtype: int64

In [104]:
for text in df["payment_behaviour"].value_counts().index:
    if re.search(pattern, text):
        print("Matched value:", text)
    else:
        print("Unmatched value:", text)

Unmatched value: Low_spent_Small_value_payments
Unmatched value: High_spent_Medium_value_payments
Unmatched value: High_spent_Large_value_payments
Unmatched value: Low_spent_Medium_value_payments
Unmatched value: High_spent_Small_value_payments
Unmatched value: Low_spent_Large_value_payments


### Type Of Loan Feature

In [105]:
df["type_of_loan"].isna().sum()

11406

In [106]:
df.loc[df["occupation"]=="Writer"]["type_of_loan"].nunique()

558

In [107]:
df.loc[df["type_of_loan"].notna()]["num_of_loan"].value_counts()

num_of_loan
3        14386
2        14249
4        14016
1        10083
6         7405
7         6930
5         6864
9         3542
-100      3459
8         3035
2_         782
4_         727
3_         718
1_         523
7_         414
6_         398
5_         332
9_         160
8_         156
1150         4
430          3
1480         3
1228         3
1217         2
141          2
58           2
1236         2
661          2
773          2
936          2
31           2
229          2
95           2
404          2
1354         2
697          2
23           2
898          2
875          2
1241         2
1259         2
955          2
288          2
1463         2
1181         2
855          2
1209         2
192          2
290          2
1353         2
284          2
733          2
466          2
1214         2
1127         2
49           2
251          2
217          2
1017         2
1365         2
1464         2
352          2
1412         2
1054         1
365          1
1023         

##### ***Let's see if the value in the "num of loan" column is related to the number of elements of the list in "type of loan"***

In [108]:
df[df["num_of_loan"] == "3"]["type_of_loan"].sample(5)

70438        Not Specified, Student Loan, and Student Loan
57683        Student Loan, Auto Loan, and Home Equity Loan
92744    Credit-Builder Loan, Debt Consolidation Loan, ...
9999       Student Loan, Payday Loan, and Home Equity Loan
58398     Home Equity Loan, Payday Loan, and Not Specified
Name: type_of_loan, dtype: object

In [109]:
df.loc[[53636]]["type_of_loan"].tolist()

['Credit-Builder Loan, Home Equity Loan, and Not Specified']

In [110]:
df.loc[[23728]]["type_of_loan"].tolist()

['Credit-Builder Loan, Not Specified, and Credit-Builder Loan']

#### ***As I see above, record in the "num_of_loan" feature has same value with number of element of list in "type_of_loan" feature*** 

In [111]:
df[df["num_of_loan"] == "1150"]

,id,customer_id,month,name,age,ssn,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,type_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
61898,0x180b0,CUS_0x96b3,March,Huw Jonesd,36.000,242-10-5844,Lawyer,61181.88,5192.490,7,6,18,1150,"Not Specified, Debt Consolidation Loan, Mortga...",30,25,3.53,8.000,Bad,2245.8,32.929,178,NM,241.570,286.422,High_spent_Small_value_payments,251.256,Poor
76482,0x1d624,CUS_0x960e,March,Jessicax,17.000,087-18-7812,Engineer,9822.095,785.508,10,8,19,1150,"Personal Loan, Student Loan, Not Specified, Pa...",35,18,31.11,8.000,Bad,2813.68,25.091,39,Yes,22.508,64.836,Low_spent_Small_value_payments,281.207,Poor
94211,0x23e05,CUS_0x7152,April,Costaf,14.000,399-91-2074,Doctor,49716.16,3925.013,4,3,20,1150,"Credit-Builder Loan, Not Specified, Personal L...",17,21,12.81,8.000,Standard,1490.78,31.310,235,Yes,240.333,72.390,High_spent_Medium_value_payments,319.778,Standard
95130,0x24368,CUS_0x12a7,March,Luciat,37.000,145-72-2113,Media_Manager,26450.59,2079.216,5,6,20,1150,"Debt Consolidation Loan, Home Equity Loan, Aut...",24,18,11.8,1.000,Standard,952.9,39.081,339,No,73.670,83.738,High_spent_Small_value_payments,310.513,Standard


In [112]:
df["num_of_loan"] = df["num_of_loan"].apply(lambda x : x.rstrip("_").lstrip("-"))

In [113]:
df["num_of_loan"] = df["num_of_loan"].astype("int")

In [114]:
df.loc[df["type_of_loan"].isna()]["num_of_loan"].value_counts()

num_of_loan
0       10928
100       417
911         2
50          2
1035        1
1347        1
193         1
995         1
1451        1
654         1
153         1
1369        1
501         1
143         1
1416        1
1070        1
238         1
103         1
172         1
252         1
1444        1
330         1
394         1
859         1
1227        1
336         1
940         1
1447        1
881         1
538         1
999         1
831         1
1384        1
621         1
1419        1
1430        1
1382        1
437         1
372         1
126         1
1074        1
288         1
378         1
652         1
33          1
439         1
1296        1
773         1
415         1
275         1
350         1
507         1
1129        1
1027        1
897         1
546         1
215         1
1478        1
83          1
18          1
801         1
Name: count, dtype: int64

In [115]:
df.loc[df["type_of_loan"].notna(),"type_of_loan"] = df.loc[df["type_of_loan"].notna()]["type_of_loan"].apply(lambda x : x.split(","))

### filled each row with number of elemnt of list in the "Type_of_Loan" feature 

In [116]:
for row in df.index :
    if not df.loc[row].isna().any() :
        num = len(df.loc[row]["type_of_loan"]) 
        df.at[row, "num_of_loan"] = num

In [117]:
df["num_of_loan"].replace(100,0,inplace = True )

In [118]:
df.loc[(df["type_of_loan"].isna()) & (df["num_of_loan"]==0),"type_of_loan"] = "new_credit"

In [119]:
df.loc[df["type_of_loan"].notna()]["num_of_loan"].unique()

array([4, 1, 3, 0, 2, 7, 5, 6, 8, 9])

In [120]:
df.loc[df["type_of_loan"].isna()]["num_of_loan"].nunique()

59

In [121]:
df["type_of_loan"].isna().sum()

61

#### I tried to correct the 2,3,4 digit meaningless, outlier values in the "num_of_loan" column with the "type_of_loan" column, but I could not find a way for the values that appeared as NaN in the "type_of_loan" column and such outliers in the "num_of_loan" column, and there are 61 rows in the 100.000 rows data. I overlooked these rows and decided to drop it.

In [122]:
nan_idx = df.loc[df["type_of_loan"].isna()].index

In [123]:
df.drop(nan_idx,inplace =True)

In [124]:
df["num_of_loan"].unique()

array([4, 1, 3, 0, 2, 7, 5, 6, 8, 9])

In [125]:
df["type_of_loan"].isna().sum()

0

In [126]:
df["annual_income"] = df["annual_income"].apply(lambda x : x.rstrip("_").lstrip("_") )

In [127]:
df["num_of_delayed_payment"] = df["num_of_delayed_payment"].apply(lambda x : x.rstrip("_").lstrip("_") )

In [128]:
df["outstanding_debt"] = df["outstanding_debt"].apply(lambda x : x.rstrip("_").lstrip("_") )

In [129]:
df.drop(["id","customer_id","name","ssn","type_of_loan"],inplace =True,axis =1)

In [130]:
df.shape

(99935, 23)

In [131]:
df.isna().sum()

month                       0
age                         0
occupation                  0
annual_income               0
monthly_inhand_salary       0
num_bank_accounts           0
num_credit_card             0
interest_rate               0
num_of_loan                 0
delay_from_due_date         0
num_of_delayed_payment      0
changed_credit_limit        0
num_credit_inquiries        0
credit_mix                  0
outstanding_debt            0
credit_utilization_ratio    0
credit_history_age          0
payment_of_min_amount       0
total_emi_per_month         0
amount_invested_monthly     0
payment_behaviour           0
monthly_balance             0
credit_score                0
dtype: int64

In [132]:
matched_re(df)

Total matches in column month: 0
Total matches in column age: 0
Total matches in column occupation: 0


Total matches in column annual_income: 0
Total matches in column monthly_inhand_salary: 0


Total matches in column num_bank_accounts: 0
Total matches in column num_credit_card: 0
Total matches in column interest_rate: 0


Total matches in column num_of_loan: 0
Total matches in column delay_from_due_date: 0
Total matches in column num_of_delayed_payment: 0


Total matches in column changed_credit_limit: 0
Total matches in column num_credit_inquiries: 0


Total matches in column credit_mix: 0
Total matches in column outstanding_debt: 0


Total matches in column credit_utilization_ratio: 0
Total matches in column credit_history_age: 0
Total matches in column payment_of_min_amount: 0


Total matches in column total_emi_per_month: 0
Total matches in column amount_invested_monthly: 0


Total matches in column payment_behaviour: 0
Total matches in column monthly_balance: 0


Total matches in column credit_score: 0
Total matches in dataframe: 0 


#### ***Last version of dataset***

In [133]:
df = pd.read_csv("/kaggle/input/last-version-of-dataset/dataset_final_30_04.csv") 

In [134]:
df.isna().sum()

id                          0
customer_id                 0
month                       0
name                        0
age                         0
ssn                         0
occupation                  0
annual_income               0
monthly_inhand_salary       0
num_bank_accounts           0
num_credit_card             0
interest_rate               0
num_of_loan                 0
type_of_loan                0
delay_from_due_date         0
num_of_delayed_payment      0
changed_credit_limit        0
num_credit_inquiries        0
credit_mix                  0
outstanding_debt            0
credit_utilization_ratio    0
credit_history_age          0
payment_of_min_amount       0
total_emi_per_month         0
amount_invested_monthly     0
payment_behaviour           0
monthly_balance             0
credit_score                0
dtype: int64

In [135]:
df.drop(["id","customer_id","name","ssn","type_of_loan"],axis=1,inplace = True)

In [136]:
df.sample(10)

,month,age,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
2867,April,18.000,Engineer,15119.56,1209.963,6,5,29,6,29,18,21.630,11.000,Bad,3154.07,41.279,154,Yes,58.211,66.176,Low_spent_Small_value_payments,286.609,Standard
35074,March,19.000,Mechanic,17414.3,1285.192,3,7,34,5,25,11,15.920,6.000,Standard,2500.7,31.370,179,Yes,61.170,72.679,Low_spent_Small_value_payments,284.671,Poor
24937,February,48.000,Media_Manager,19042.55,1716.879,3,7,15,12,10,16,13.300,1.000,Standard,36.36,39.233,267,Yes,0.000,231.084,Low_spent_Small_value_payments,230.604,Standard
45220,May,39.000,Teacher,83418.18,7113.515,0,4,6,1,16,6,11.900,2.000,Good,972.51,32.115,361,No,59.584,126.233,High_spent_Large_value_payments,765.535,Standard
93357,June,31.000,Manager,70543.77,5986.648,5,5,16,4,16,15,17.680,7.000,Standard,1395.86,28.623,290,NM,218.011,258.243,High_spent_Large_value_payments,577.307,Standard
5436,May,18.000,Journalist,71368.6,5844.383,6,3,6,12,19,15,10.880,8.000,Standard,1161.1,31.816,335,Yes,0.000,211.303,Low_spent_Medium_value_payments,653.135,Standard
86154,March,19.000,Musician,24293.22,1906.435,4,7,14,3,5,18,11.660,2.000,Standard,62.26,38.396,255,No,55.439,72.801,High_spent_Medium_value_payments,312.403,Standard
38216,January,18.000,Journalist,9092.87,477.739,6,6,16,5,11,17,15.800,8.000,Standard,2216.91,35.479,92,Yes,30.733,50.045,Low_spent_Medium_value_payments,256.995,Standard
48198,July,41.000,Engineer,15957.41,1039.784,8,5,7,3,18,19,16.330,4.000,Standard,1269.87,37.501,129,Yes,25.049,73.459,Low_spent_Small_value_payments,295.471,Standard
83952,January,14.000,Manager,61922.24,5418.187,6,9,18,9,25,19,23.280,12.000,Bad,4678.77,25.661,40,Yes,336.015,131.642,High_spent_Large_value_payments,314.162,Poor


In [137]:
df = df.replace("_","",regex=True)

In [138]:
df.loc[[67491]]

,month,age,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
67491,April,22.000,Teacher,104849.82,8744.485,4,6,18,12,29,12,18.280,1.000,Standard,1439.21,27.590,257,Yes,0.000,168.072,HighspentLargevaluepayments,946.376,Good


In [139]:
df.shape

(99935, 23)

In [140]:
df.sample(10)

,month,age,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
99936,January,47.000,Developer,22620.79,1722.066,7,3,9,12,27,18,5.310,2.000,Standard,642.46,31.606,351,No,0.000,56.073,HighspentMediumvaluepayments,366.134,Standard
27462,July,38.000,Writer,55139.82,4703.985,3,4,5,4,28,10,8.400,11.000,Standard,95.73,35.019,303,Yes,110.392,363.508,LowspentSmallvaluepayments,286.499,Standard
91030,July,25.000,Lawyer,129903.0,10705.250,4,6,8,2,6,8,8.160,4.000,Good,585.74,28.818,199,No,170.012,234.981,HighspentLargevaluepayments,941.536,Standard
67020,May,18.000,Engineer,7632.555,476.046,7,8,32,6,12,10,15.260,7.000,Standard,1401.9,39.924,149,Yes,34.929,26.260,LowspentSmallvaluepayments,276.416,Poor
21041,February,23.000,Architect,99663.0,8551.250,5,3,7,4,15,0,11.500,4.000,Good,96.62,27.185,245,NM,293.318,442.242,LowspentMediumvaluepayments,399.564,Good
22057,February,26.000,Scientist,9846.615,826.551,8,5,19,8,26,17,1.550,12.000,Bad,4121.85,37.470,76,Yes,33.196,31.295,LowspentLargevaluepayments,247.918,Poor
94340,May,21.000,Lawyer,68751.75,5468.312,6,3,10,12,25,14,11.610,1.000,Good,64.47,38.421,357,No,0.000,206.385,LowspentSmallvaluepayments,630.446,Good
93680,January,38.000,Teacher,20528.59,1900.716,6,7,32,9,61,15,28.490,12.000,Bad,3578.54,34.261,63,Yes,127.902,131.368,HighspentSmallvaluepayments,190.802,Poor
76931,April,51.000,Teacher,30961.62,2283.135,5,7,11,12,10,6,6.520,1.000,Good,1304.15,27.978,309,No,0.000,254.709,LowspentSmallvaluepayments,263.604,Poor
94449,February,27.000,Mechanic,34428.72,2923.060,4,3,9,5,6,18,18.480,8.000,Standard,478.11,39.874,92,NM,130.019,180.179,HighspentSmallvaluepayments,242.108,Standard


In [141]:
df.to_csv("cleaned_creditscore_dataset.csv",index = False)

In [142]:
df = pd.read_csv('/kaggle/working/cleaned_creditscore_dataset.csv')
df.head()

,month,age,occupation,annual_income,monthly_inhand_salary,num_bank_accounts,num_credit_card,interest_rate,num_of_loan,delay_from_due_date,num_of_delayed_payment,changed_credit_limit,num_credit_inquiries,credit_mix,outstanding_debt,credit_utilization_ratio,credit_history_age,payment_of_min_amount,total_emi_per_month,amount_invested_monthly,payment_behaviour,monthly_balance,credit_score
0,January,23.000,Scientist,19114.120,1824.843,3,4,3,4,3,7,11.270,4.000,Good,809.980,26.823,265,No,49.575,80.415,HighspentSmallvaluepayments,312.494,Good
1,February,23.000,Scientist,19114.120,1824.843,3,4,3,4,-1,7,11.270,4.000,Good,809.980,31.945,265,No,49.575,118.280,LowspentLargevaluepayments,284.629,Good
2,March,23.000,Scientist,19114.120,1824.843,3,4,3,4,3,7,11.270,4.000,Good,809.980,28.609,267,No,49.575,81.700,LowspentMediumvaluepayments,331.210,Good
3,April,23.000,Scientist,19114.120,1824.843,3,4,3,4,5,4,6.270,4.000,Good,809.980,31.378,268,No,49.575,199.458,LowspentSmallvaluepayments,223.451,Good
4,May,23.000,Scientist,19114.120,1824.843,3,4,3,4,6,4,11.270,4.000,Good,809.980,24.797,269,No,49.575,41.420,HighspentMediumvaluepayments,341.489,Good
